In [ ]:
# Install or update openai modules 
!pip install openai python-dotenv mutagen
!pip install --upgrade openai
!pip install edge-tts
!pip install pygame
!pip install python-dotenv
!pip install pydub
!pip install beautifulsoup4

In [ ]:
# Load Modules
from openai import OpenAI
import IPython
from datetime import datetime
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, APIC, TPE1, TALB, TCON
import sys
from dotenv import load_dotenv
import os

import asyncio
import edge_tts
import pygame
from pydub import AudioSegment
from bs4 import BeautifulSoup

In [ ]:
def read_lines_from_file(file_path):
  with open(file_path, 'r') as file:
    lines = file.readlines()
  return lines

# Example usage
file_path = 'idioms.txt'  # Replace with the path to your text file
lines = read_lines_from_file(file_path)
print(lines)
print(len(lines))

# # save the idioms to a file
# with open('idioms.txt', 'w') as f:
#     for item in lines:
#         f.write("%s\n" % item)

In [ ]:

# load_dotenv()
# client = OpenAI(
# )

# 获取 API 密钥
api_key = 'xxx'
# 初始化 OpenAI 客户端
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

In [ ]:
def get_base_text(idiom):
  role_definition = """
  你是我的儿童英语教练。我将提供一个带有编号的 idiom，你要帮我把他翻译成合适儿童理解的中文释疑。
  而后，讲解一下这个 idiom 的来历=，以及适用的场合，尽量口语化，儿童容易理解。
  再然后，你要为这个 idiom 提供 3 个美式英文例句，尽量使用日常词汇，儿童容易理解，不要过长。
  
  [Overall Rules to follow]
    1. Do not compress your responses.

  参照以下例子（markdown）作为输出格式(follow the Overall Rules to follow)，
  其中，请格外注意 span 和 audio html 标签的正确，以及，弯引号外与汉字之间应该有一个空格：
  
```  
## 1. Hello

**释义**

“打招呼” 或 “你好”。

Hello 这个词啊，很久以前人们用它来叫别人的时候，就像我们说“喂”一样。后来呢，大家就习惯用它来打招呼了，就像我们现在说“你好”一样。你可以用 Hello 来跟朋友、老师或者家人打招呼，比如你早上起床见到妈妈，就可以说 Hello。或者你接到电话，也可以说 Hello。

Hello, this word was used by people long ago to call out to others, just like we say "hey" in Chinese. Then, everyone got used to using it as a greeting, just like we say "hello" now. You can use Hello to greet your friends, teachers, or family members. For example, when you wake up in the morning and see your mom, you can say Hello. Or when you answer the phone, you can also say Hello.

**Examples**

- <span lang="en">Hello, Tommy!</span> <span lang="cn">你好，汤米！</span><audio controls><source src="audio/001.1.mp3" type="audio/mpeg"></audio>
- <span lang="en">Hello, can I have a cookie?</span> <span lang="cn">你好，我可以吃一块饼干吗？</span><audio controls><source src="audio/001.2.mp3" type="audio/mpeg"></audio>
- <span lang="en">Hello, it's nice to meet you! </span> <span lang="cn">你好，很高兴见到你！</span><audio controls><source src="audio/001.3.mp3" type="audio/mpeg"></audio>

```
"""
  
  user_prompt = idiom
  
  # how many versions needed.
  number_of_choices = 1
  
  rspd_translation = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
      {
          "role": "system", 
           "content": role_definition
      },
      {
          "role": "user", 
          "content": user_prompt
      }
    ],
    n = number_of_choices 
  )
  return rspd_translation.choices[0].message.content

markdown_text = get_base_text("9. Sorry")
print(markdown_text)

In [ ]:
def get_example_list(text):
  soup = BeautifulSoup(text, 'html.parser')
  english_spans = soup.find_all('span', lang='en')
  return [span.get_text() for span in english_spans]

example_list = get_example_list(markdown_text)
print(example_list)

In [ ]:
def add_zero(n):
  if n < 10: 
    return f'00{n}'
  if n >9 and n <100: 
    return f'0{n}'
  if n >= 100: 
    return f'{n}'

print(add_zero(400))

In [ ]:
!pip install requests

In [ ]:
import base64
import json
import uuid
import requests
import random

# 填写平台申请的appid, access_token以及cluster
appid = "xxx"
access_token= "xxx"
cluster = "xxx"

host = "openspeech.bytedance.com"
api_url = f"https://{host}/api/v1/tts"

header = {"Authorization": f"Bearer;{access_token}"}

def add_zero(number):
    return str(number).zfill(2)

def get_example_sentence_audio(example_sentence, number, order):
    text = example_sentence
    n = add_zero(number)
    
    voice_list = ['BV506_streaming', 'BV702_streaming', 'BV421_streaming', 'BV503_streaming']
    voice_performer = random.choice(voice_list)
    
    speech_file_path = f'audio/{n}.{order}.mp3'
    
    request_json = {
        "app": {
            "appid": appid,
            "token": access_token,
            "cluster": cluster
        },
        "user": {
            "uid": "388808087185088"
        },
        "audio": {
            "voice_type": voice_performer,
            "encoding": "mp3",
            "speed_ratio": 1.0,
            "volume_ratio": 1.0,
            "pitch_ratio": 1.0,
        },
        "request": {
            "reqid": str(uuid.uuid4()),
            "text": text,
            "text_type": "plain",
            "operation": "query",
            "with_frontend": 1,
            "frontend_type": "unitTson"
        }
    }

    try:
        resp = requests.post(api_url, data=json.dumps(request_json), headers=header)
        print(f"resp body: \n{resp.json()}")
        if "data" in resp.json():
            data = resp.json()["data"]
            with open(speech_file_path, "wb") as file_to_save:
                file_to_save.write(base64.b64decode(data))
        print(f'The audio of {n} is saved...')
    except Exception as e:
        print(f"Error: {e}")

# 示例调用
example_list = ['example sentence 1', 'example sentence 2']
for e in example_list:
    get_example_sentence_audio(e.replace("*", ""), 38, example_list.index(e)+1)

In [ ]:
# import random
# def get_example_sentence_audio(example_sentence, number, order):
#   text = example_sentence
#   n = add_zero(number)
  
#   voice_list = ['alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer']
#   voice_performer = random.choice(voice_list)
#   # alloy, echo, fable, onyx, nova, and shimmer, the last two of which are femail voices.
  
#   speech_file_path = f'audio/{n}.{order}.mp3'
  
#   rspd_audio = client.audio.speech.create(
#     model="tts-1",
#     voice=voice_performer,
#     input=text
#   ) 
  
#   rspd_audio.stream_to_file(speech_file_path)
#   print(f'The audio of {n} is saved...')


# for e in example_list:
#   get_example_sentence_audio(e.replace("*", ""), 38, example_list.index(e)+1)


In [ ]:
import re

def extract_number_from_string(s):
    match = re.search(r'\d+', s)
    if match:
        return int(match.group())
    else:
        return None  # Return None if no number is found

# Example usage
input_string = "44. Costs an arm and a leg"
number = extract_number_from_string(input_string)
print(number)  # Output: 44

In [ ]:
with open("book.md", 'w') as file:

  for l in lines:
    markdown_text = get_base_text(str(lines.index(l)+1) + '. ' + l.strip())  
    print(markdown_text)
    file.write(markdown_text)
    
    example_list = get_example_list(markdown_text)
    for e in example_list:
      get_example_sentence_audio(e.replace("*", ""), lines.index(l)+1, example_list.index(e)+1)
    
    file.write("\n\n")
    
    print(l)
          
print("All done!")
  
    

In [ ]:
from pydub import AudioSegment
import os

# combine audios into larger chunck
combined = AudioSegment.empty()

for i in range(1351, 1356):
  for j in range(1,4):
    print(i, j)
    if j == 1:
      combined += AudioSegment.from_file("1-sec.mp3")
      combined += AudioSegment.from_file("Funk.mp3")
      combined += AudioSegment.from_file("1-sec.mp3")
    combined += AudioSegment.from_file(f"audio/{add_zero(i)}.{j}.mp3")
    combined += AudioSegment.from_file(f"3-sec.mp3")
    if (i % 10 == 0 or i == 1355) and j == 3:
      combined += AudioSegment.from_file("ending.mp3")
      combined.export(f"{int(i/10)-1}1-{i}.mp3")
      combined = AudioSegment.empty()

In [ ]:
from pydub import AudioSegment

def create_silence_mp3(duration_seconds, output_file):
    # Create a silent audio segment
    silence = AudioSegment.silent(duration=duration_seconds * 1000)  # duration in milliseconds
    # Export the silent audio segment as an MP3 file
    silence.export(output_file, format="mp3")

# Duration of silence in seconds
n = 10  # Change this to the desired number of seconds
output_file = "1-sec.mp3"  # Output file name

create_silence_mp3(1, output_file)

print(f"{n} seconds of silence have been saved to {output_file}.")

In [ ]:
def add_metadata(mp3_file, image_file, artist, album, genre):
    try:
        audio = MP3(mp3_file, ID3=ID3)

        # Add ID3 tag if it doesn't exist
        try:
            audio.add_tags()
        except:
            pass

        # Add artist name
        audio.tags.add(TPE1(encoding=3, text=artist))

        # Add album name
        audio.tags.add(TALB(encoding=3, text=album))

        # Add genre
        audio.tags.add(TCON(encoding=3, text=genre))

        # Add artwork
        with open(image_file, 'rb') as albumart:
            audio.tags.add(APIC(
                encoding=3,          # 3 is for utf-8
                mime='image/jpeg',   # image/jpeg or image/png
                type=3,              # 3 is for the cover image
                desc=u'Cover',
                data=albumart.read()
            ))

        audio.save()
        print(f"Metadata added to {mp3_file}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
mp3_files = [file for file in os.listdir('audio/') if file.endswith('.mp3')]
for f in mp3_files:
  image_file = 'Cover.png'
  artist = '@Jasonbai & Deepseek/volcengine'
  album = 'Common English Words for Children'
  genre = 'SPEECH'
  add_metadata(f"audio/{f}", image_file, artist, album, genre)